In [2]:
import pandas as pd



In [11]:
import pandas as pd
import zipfile


zip_file_path = '/content/Moies.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('extracted_folder')


credits_df = pd.read_csv('extracted_folder/tmdb_5000_credits.csv')
movies_df = pd.read_csv('extracted_folder/tmdb_5000_movies.csv')


merged_df = pd.merge(movies_df, credits_df, on='title', how='inner')


print(merged_df)


         budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4804     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4805       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4806          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4807          0                                                 []   
4808          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

In [13]:
import pandas as pd

merged_df = pd.merge(credits_df, movies_df, on='title', how='inner')

print(merged_df)


      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                                   cast  \
0     [{"cast_id": 242, "character": "Jake Sully", "...   
1     [{"cast_id": 4, "character": "Captain Jack Spa...   
2     [{"cast_id": 1, "character": "James Bond", "cr...   
3     [{"cast_id": 2, "character": "Bruce W

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


features = ['title', 'genres', 'keywords', 'cast', 'crew', 'overview']


merged_df[features] = merged_df[features].fillna('')


merged_df['combined_features'] = merged_df['genres'] + ' ' + merged_df['keywords'] + ' ' + merged_df['cast'] + ' ' + merged_df['crew'] + ' ' + merged_df['overview']


tfidf_vectorizer = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['combined_features'])


cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(title):
    try:
        idx = merged_df.index[merged_df['title'] == title].tolist()[0]
    except IndexError:
        print(f"Movie with title '{title}' not found.")
        return pd.Series()  # Return an empty Series if the movie is not found

    sim_scores = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies (excluding itself)

    movie_indices = [i[0] for i in sim_scores]
    return merged_df['title'].iloc[movie_indices]


movie_title = 'Avatar'
recommendations = get_recommendations(movie_title)
print(f"Recommended movies for '{movie_title}':")
print(recommendations)


Movie with title 'Avatar' not found.
Recommended movies for 'Avatar':
Series([], dtype: float64)


<ipython-input-23-32448416d701>:32: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()  # Return an empty Series if the movie is not found


In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


features = ['title', 'genres', 'keywords', 'cast', 'crew', 'overview']

merged_df[features] = merged_df[features].fillna('')


merged_df['combined_features'] = merged_df['genres'] + ' ' + merged_df['keywords'] + ' ' + merged_df['cast'] + ' ' + merged_df['crew'] + ' ' + merged_df['overview']


tfidf_vectorizer = TfidfVectorizer(stop_words='english')


tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['combined_features'])

cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)


def get_recommendations(movie_title):
    try:
        idx = merged_df.index[merged_df['title'] == movie_title].tolist()[0]
    except IndexError:
        print(f"Movie with title '{movie_title}' not found.")
        return pd.Series()  # Return an empty Series if the movie is not found

    sim_scores = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get top 10 similar movies (excluding itself)

    movie_indices = [i[0] for i in sim_scores]
    return merged_df['title'].iloc[movie_indices]


selected_movie_title = 'Avatar'
recommendations = get_recommendations(selected_movie_title)

print(f"Recommended movies for '{selected_movie_title}':")
print(recommendations)


Movie with title 'Avatar' not found.
Recommended movies for 'Avatar':
Series([], dtype: float64)


<ipython-input-24-c2b31c53285c>:32: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()  # Return an empty Series if the movie is not found
